In [1]:
version = "v1.12.052622.1"

# SIADS 543 Assignment 3: Text representations, topic modeling and word embeddings

In this week's assignment you'll gain experience applying topic modeling and other latent variable estimation methods. We'll focus on textual data, continuing to work with vectorizers and related text representations like embeddings.

All questions in this assignment are auto-graded. Some parts ask you a short question or two about on the results: these are meant to encourage you to reflect on the outcomes, but do not need to be included as part of your graded submission.

In [2]:
# First import some necessary libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

# Suppress all warnings
import warnings

warnings.filterwarnings("ignore")

np.set_printoptions(precision=3)

#### Here are some useful utility functions to use with this assignment.

In [3]:
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

# display_topics:  example showing how to take the model components generated by LDA or NMF
# and use them to dump the top words by weight for each topic.
def display_topics(model, feature_names, num_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic %d:" % (topic_idx))
        print(
            " ".join(
                [feature_names[i] for i in topic.argsort()[: -num_top_words - 1 : -1]]
            )
        )


# load_newsgroup_documents: prepare training and test data from the 20newsgroups dataset
def load_newsgroup_documents():
    # The Coursera environment must be self-contained and so APIs that do external fetching
    # aren't allowed. So we use pickle files that can be stored locally instead of the following
    # API calls.
    # dataset_train   = fetch_20newsgroups(subset = 'train', shuffle=True, random_state=42, remove=('headers', 'footers', 'quotes'))
    # dataset_test    = fetch_20newsgroups(subset = 'test', shuffle=True, random_state=42, remove=('headers', 'footers', 'quotes'))

    pickle_train_data = open("./assets/20newsgroups_train_data.pickle", "rb")
    pickle_train_labels = open("./assets/20newsgroups_train_labels.pickle", "rb")
    documents_train = pickle.load(pickle_train_data)
    labels_train = pickle.load(pickle_train_labels)
    pickle_train_data.close()
    pickle_train_labels.close()

    return documents_train, labels_train


# load the dataset for future use....
documents_train, labels_train = load_newsgroup_documents()

## Question 1 (20 points)  The choice of text processing can impact final classification performance.

There are many different parameter settings for Vectorizer objects in scikit-learn. Small changes in these settings can result in very different text representations and significant changes in final classifier accuracy. For this question you'll train a commonly-used type of text classifier, Multinomial Naive Bayes, using three different input representations for text, to see the effect of different parameter choices on classifier training set accuracy.

Follow these steps:
1. Create a TfidfVectorizer object (let's call it A) with the following settings:

    `max_features = 10000, # only top 10k by freq`
    
    `lowercase = False, # keep capitalization`
    
    `ngram_range = (1,2), # include 2-word phrases`
    
    `min_df=10,  # note: absolute count of documents`
    
    `max_df=0.95,   # note: % of docs in collection`
    
    `stop_words='english'`
    
    
2. Create a CountVectorizer object (let's call it B) with the same settings:

    `max_features = 10000, # only top 10k by freq`
    
    `lowercase = False, # keep capitalization`
    
    `ngram_range = (1,2), # include 2-word phrases`
    
    `min_df=10,  # note: absolute count of doc`
    
    `max_df=0.95,   # note: % of docs`
    
    `stop_words='english'`
    
3. Create a TfidfVectorizer object (let's call it C) with the settings:

    `max_features = 10000, # only top 10k by freq`
    
    `lowercase = False, `
    
    `ngram_range = (1,2), `
    
    `min_df=200,  # note: absolute count of docs`
    
    `max_df=0.95  # note: % of docs` 
    
    
4. Using the training data `documents_train`, along with the ground truth labels `labels_train`, train three Naive Bayes classifiers, corresponding to choices A, B, and C of vectorizer.

5. Normally we'd compute the accuracy of these classifiers on a test set, but for this question we're interested more in the potential upper bound on performance that is achievable with text representation choices A, B, or C.  Thus you should compute, for each of the three classifiers, the accuracy on the *training set*.

6. Your function should return these three accuracy scores as a tuple with three floats: (accuracy_A, accuracy_B, accuracy_C).

It is instructive to examine the difference in accuracy across the three different representations. 


In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics

def answer_text_processing():
    
    A = TfidfVectorizer(max_features = 10000, # only top 10k by freq
                        lowercase = False, # keep capitalization
                        ngram_range = (1,2), # include 2-word phrases
                        min_df=10,  # note: absolute count of documents
                        max_df=0.95,   # note: % of docs in collection
                        stop_words='english') # default English stopwords

    B = CountVectorizer(max_features = 10000, # only top 10k by freq
                        lowercase = False, # keep capitalization
                        ngram_range = (1,2), # include 2-word phrases
                        min_df=10,  # note: absolute count of doc
                        max_df=0.95,   # note: % of docs
                        stop_words='english')

    C = TfidfVectorizer(max_features = 10000, # only top 10k by freq
                        lowercase = False,
                        ngram_range = (1,2),
                        min_df=200,  # note: absolute count of docs
                        max_df=0.95)  # note: % of docs # default English stopwords
    
    naive_bayes_classifier = MultinomialNB()

    A_documents = A.fit_transform(documents_train)
    A_fit = naive_bayes_classifier.fit(A_documents, labels_train)
    A_pred = A_fit.predict(A_documents)
    A_score = metrics.accuracy_score(labels_train, A_pred)

    B_documents = B.fit_transform(documents_train)
    B_fit = naive_bayes_classifier.fit(B_documents, labels_train)
    B_pred = B_fit.predict(B_documents)
    B_score = metrics.accuracy_score(labels_train, B_pred)

    C_documents = C.fit_transform(documents_train)
    C_fit = naive_bayes_classifier.fit(C_documents, labels_train)
    C_pred = C_fit.predict(C_documents)
    C_score = metrics.accuracy_score(labels_train, C_pred)

    result = A_score, B_score, C_score

    return result

In [5]:
stu_ans = answer_text_processing()

assert isinstance(stu_ans, tuple), "Q1: Your function should return a tuple."
assert len(stu_ans) == 3, "Q1: Your tuple should contain three floats."

for i, item in enumerate(stu_ans):
    assert isinstance(
        item, (float, np.floating)
    ), f"Q1: Your answer at index {i} should be a float number. "

# Some hidden tests

del stu_ans

## Question 2 (30 points). Latent Semantic Indexing and the vocabulary gap.

One of the original motivations for Latent Semantic Indexing was overcoming the `vocabulary gap` in information retrieval.
A query like `economic budget` should match strongly against text like `government spending on the economy` even though they don't have any exact keywords in common.

In this question we'll create a demonstration of the power of Latent Semantic Indexing to do semantic matching. In the first part, you'll run LSI and use the reduced document matrix to do semantic matching of a query against other text that has no terms explicitly in common.

In the second part, you'll see how this semantic matching is happening by computing the related terms that are included a query expanded using LSI's latent topics.

### Part 2.1 (15 points) Use the reduced document matrix from LSI to do semantic matching of a query against a document.

As a first step, run the code below that we've provided that creates a tf.idf vectorizer and applies it to the 20newsgroups training set. It also runs LSI (in reality a TruncatedSVD) with a latent space of 200 dimensions.

Suppose we have a query "economic budget" that has the tf.idf vector $q$, with shape 1 x num_terms. We can obtain this vector simply by using vectorizer.transform on the text. Think of the matrix $U_k$ as the super operator that converts from original term space to latent semantic space. To expand text $q$ with related terms according to LSI, compute the expanded query $q_k$ using the formula 

$q_k = q \cdot (U_k\Sigma^{-1}_k)$

With this formula, you'll "expand" both the query and the document vectors to add related terms, and then compute the similarity match between them.

Let's walk through these steps (**Note** that in matrix math dimensions and hence order of operations matters!).  
1. The _reduced term matrix_, $U_k$, is _multiplied_ (* operator) with the inverse of matrix $\Sigma_k$, the LSI.singular_values_ matrix. **Note** that $\Sigma_k$ is raised to the power of negative one $\Sigma^{-1}_k$. The reason we invert $\Sigma_k$ is that there is no division operation with matrices so we invert and multiply!
 - Think of this step as forming a normalized scaler for the LSI latent factor weights (the 'topics').  


2. The vectorized query matrix $q$ (or document $d$) is then dotted (dot-product) with the result of $U_k \Sigma^{-1}_k$.

For this question, use cosine similarity to compute the similarity match between any two pieces of text, no matter what their vector representation.

With the formula above, consider the query `"economic budget"` being matched against the (very) short document `"government spending on the economy"`.

Your function should return a tuple of two floats: the cosine similarity score (from sklearn.metrics.pairwise) of (a) the original query and document vectors and (b) the LSI-expanded query and document vectors using the method above.

Did LSI help overcome the vocabulary gap?

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

### Run this preamble code to run LSI. We've also given the line of code that gets the resulting U matrix.
tfidf_vectorizer = TfidfVectorizer(
    ngram_range=(1, 1), min_df=2, max_df=0.95, stop_words="english", max_features=10000
)  # default English stopwords

tfidf_documents = tfidf_vectorizer.fit_transform(documents_train)
tfidf_feature_names = tfidf_vectorizer.get_feature_names()

# LSI does truncated SVD on the document-term matrix of tf.idf term-weights.
# The matrix we got back from the vectorizer is a
# document-term matrix, i.e. one row per document.
n_topics = 200
lsi = TruncatedSVD(n_components=n_topics, random_state=0)

# To match the examples and development of LSI in
# our lectures, we're going to
# take the transpose of the document-term matrix to give
# TruncatedSVD the term-document matrix as input.

# This is the matrix U_k:  num_term_features x num_topics
reduced_term_matrix = lsi.fit_transform(np.transpose(tfidf_documents))

In [7]:
from sklearn.metrics.pairwise import cosine_similarity

def answer_semantic_similarity_a():
    
    q = tfidf_vectorizer.transform(['economic budget'])
    d = tfidf_vectorizer.transform(['government spending on the economy'])
    first_step = cosine_similarity(q,d).item(0) #0!

    Uk_Sigma_q = reduced_term_matrix * (1/lsi.singular_values_)
    new_q = np.dot(q.toarray(), Uk_Sigma_q)

    Uk_Sigma_d = reduced_term_matrix * (1/lsi.singular_values_)
    new_d = np.dot(d.toarray(), Uk_Sigma_d)
    
    second_step = cosine_similarity(new_q, new_d).item(0)

    result = first_step, second_step

    return result

In [8]:
stu_ans = answer_semantic_similarity_a()

assert isinstance(stu_ans, tuple), "Q2a: Your function should return a tuple. "
assert len(stu_ans) == 2, "Q2a: Your tuple should contain two floats."

for i, item in enumerate(stu_ans):
    assert isinstance(
        item, (float, np.floating)
    ), f"Q2a: Your answer at index {i} should be a float number. "

# Some hidden tests

del stu_ans

### Part 2.2 (15 points): We want to understand this semantic matching ability a bit more: what terms does LSI think are similar?

To understand why the LSI-expanded vectors get the results they do, we're going to look at what the operator $U$ does to text. In particular, the term-term matrix $UU^T$ tells us the term expansion behavior of this LSI model. Think of the term-term matrix like an operator that first maps a term to the latent space L_k (using $U$), then back again from L_k to term space (using $U$ transpose). The $(i,j)$ entry of $UU^T$ is a kind of *association weight* between term $i$ and term $j$.

Write a function to get the most related terms (according to LSI) for the word "economy". To do this:

1. Compute the term-term matrix from the matrix U  (the reduced_term_matrix variable).
2. Use the term-term matrix to get the association weights of all words related to the term "economy"
3. Sort by descending weight value.
4. Your function should return the top 5 words and their weights as a list of (string, float) tuples.

Do the related terms match your subjective similarity judgment?

In [9]:
def answer_semantic_similarity_b():

    term_term_matrix = np.dot(reduced_term_matrix, np.transpose(reduced_term_matrix))
    economy_index = tfidf_vectorizer.vocabulary_['economy']
    
    top_related_term_indexes = term_term_matrix[economy_index, :].argsort()[::-1]
    result = []

    for i in range(5):
        next_term = top_related_term_indexes[i]
        next_tuple = tfidf_feature_names[next_term], term_term_matrix[economy_index, next_term]
        result.append(next_tuple)

    return result

In [10]:
stu_ans = answer_semantic_similarity_b()

assert isinstance(stu_ans, list), "Q2b: Your function should return a list. "
assert (
    len(stu_ans) == 5
), "Q2b: Your list should contain five elements (the term, score tuples)."

for i, item in enumerate(stu_ans):
    assert isinstance(item, tuple), f"Q2b: Your answer at index {i} should be a tuple. "
    assert isinstance(
        item[0], str
    ), f"Q2b: The first element of your tuple at index {i} should be a string. "
    assert isinstance(
        item[1], (float, np.floating)
    ), f"Q2b: The second element of your tuple at index {i} should be a float. "

# Some hidden tests

del stu_ans

## Question 3 (20 points) Semantic similarity: comparing your ranking with word2vec's ranking


### Part 3.1 Ranking words by yourself and then generating word2vec's ranking

First, rearrange the words in the `my_ranking` tuple based on your subjective impression of how similar they are to the word `"party"`. Encode your impression with words arranged in decreasing similarity to `"party"`; index 0 holds `"party"`, index 1 the next most similar term all the way to index 8 holding what you consider the least similar. **This is your personal subjective understanding of how similar these terms are to `"party"`.**

In [11]:
# EDIT THE FOLLOWING variable my_ranking.
#
# The target word is **'party'**, which you should keep first in the tuple. Edit the order of the rest of the words
# in my_ranking so that it reflects
# YOUR subjective ranking for how semantically similar each word is to the word 'party'. For example,
# if you think 'event' is the most similar word to 'party', it should be placed second in the list after 'party'
# and so on. Make sure you use all the words : just re-order them.
my_ranking = (
    "party",
    "event",
    "fun",
    "champagne",
    "budget",
    "bicycle",
    "vote",
    "lead",
    "election",
)

### Before proceeding:
 - The following code cell must be executed to load the pre-trained word2vec model.

In [12]:
#### We need to load the pre-trained word2vec model.
#### The result is an instance of the class W2VTransformer(size=100, min_count=1, seed=2)
#### from gensim.sklearn_api import W2VTransformer

import pickle

f = open("./assets/text8_W2V.pickle", "rb")
text8_model = pickle.load(f)
f.close()

The second part of the task is to derive the model's understanding of how similar these terms are to the string "party". One way to do this:

1. Use the word2vec `text8_model` to convert `my_ranking` into the corresponding set of word embedding vectors.
2. Determine the cosine similarity of each word embedding relative to the target embedding of `"party"`. Maintain a list of tuples in the format of `(similarity_score, word)`. Do not skip over the embedding for the target word `"party"`, the list should include the similarity to itself.
3. Sort the list in decreasing order. Remember, when sorting elements of type tuple, the first element determines the sorting order and the second element is only considered if there is a tie.
4. Create a list variable called `system_ranking` which contains only the second element of each tuple (the words). 

Your function should return a tuple in the format of `(my_ranking, system_ranking)`.

In [13]:
def answer_word2vec1():
    
    embeddings = text8_model.transform(my_ranking)

    score_list = []

    for i in range(9):
        similarity = cosine_similarity(embeddings[0].reshape(1, -1), embeddings[i].reshape(1, -1)).item()
        temp_tuple = (similarity, my_ranking[i])
        score_list.append(temp_tuple)

    sorted_scores = sorted(score_list, key = lambda x: x[0], reverse = True)
    system_ranking = [x[1] for x in sorted_scores]
    result = my_ranking, tuple(system_ranking)

    return result

In [14]:
reference_terms = (
    "party",
    "bicycle",
    "vote",
    "lead",
    "election",
    "champagne",
    "event",
    "fun",
    "budget",
)

stu_ans = answer_word2vec1()

assert isinstance(stu_ans, tuple), "Q3.1: Your function should return a tuple. "
assert (
    len(stu_ans) == 2
), "Q3.1: Your tuple should contain two elements: a tuple of strings (my_ranking), and a tuple of strings (system_ranking)."

# check my_rankings
assert isinstance(
    stu_ans[0], tuple
), "Q3.1: Your first element must be a tuple (of strings). "
assert len(stu_ans[0]) == len(
    reference_terms
), "Q3.1: Your my_rankings tuple doesn't have the expected number of terms."
assert (
    stu_ans[0][0] == reference_terms[0]
), "Q3.1: Your my_rankings tuple must have 'party' as the first term."
assert set(stu_ans[0]) == set(
    reference_terms
), "Q3.1: Your my_rankings tuple is not a permutation of the permitted terms."  # must be a permutation of the official term set

# check system_rankings
assert isinstance(
    stu_ans[1], tuple
), "Q3.1: Your second element must be a tuple (of strings). "
assert len(stu_ans[1]) == len(
    reference_terms
), "Q3.1: Your system_rankings tuple doesn't have the expected number of terms."
assert (
    stu_ans[0][0] == reference_terms[0]
), "Q3.1: Your system_rankings tuple must have 'party' as the first term."
assert set(stu_ans[1]) == set(
    reference_terms
), "Q3.1: Your system_rankings tuple is not a permutation of the permitted terms."  # must be a permutation of the official term set

# Some hidden tests

del stu_ans

### Part 3.2 Comparing rankings with Spearman correlation coefficient

Given the rankings you generated above in the format of `(my_ranking, system_ranking)`, use the [`scipy.stats.spearmanr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.spearmanr.html) function which will return a SciPy object containing the spearman correlation and the p-value of your ranking compared to the system ranking, as a measurement of how well they agree. Convert this spearman result to a tuple, and append that tuple to your previous results, i.e. it should have a format of `(my_ranking, system_ranking, spearman_tuple)`.

In [15]:
from scipy.stats import spearmanr

def answer_word2vec():

    my_ranking, system_ranking = answer_word2vec1()
    result = my_ranking, system_ranking, tuple(spearmanr(my_ranking, system_ranking))

    return result

In [16]:
reference_terms = (
    "party",
    "bicycle",
    "vote",
    "lead",
    "election",
    "champagne",
    "event",
    "fun",
    "budget",
)

stu_ans = answer_word2vec()

assert isinstance(stu_ans, tuple), "Q3.2: Your function should return a tuple. "
assert (
    len(stu_ans) == 3
), "Q3.2: Your tuple should contain three elements: a tuple of strings (my_ranking), a tuple of strings (system_ranking), a tuple with spearman output (2 floats)."

# check my_rankings
assert isinstance(
    stu_ans[0], tuple
), "Q3.2: Your first element must be a tuple (of strings). "
assert len(stu_ans[0]) == len(
    reference_terms
), "Q3.2: Your my_rankings tuple doesn't have the expected number of terms."
assert (
    stu_ans[0][0] == reference_terms[0]
), "Q3.2: Your my_rankings tuple must have 'party' as the first term."
assert set(stu_ans[0]) == set(
    reference_terms
), "Q3.2: Your my_rankings tuple is not a permutation of the permitted terms."  # must be a permutation of the official term set

# check system_rankings
assert isinstance(
    stu_ans[1], tuple
), "Q3.2: Your second element must be a tuple (of strings). "
assert len(stu_ans[1]) == len(
    reference_terms
), "Q3.2: Your system_rankings tuple doesn't have the expected number of terms."
assert (
    stu_ans[0][0] == reference_terms[0]
), "Q3.2: Your system_rankings tuple must have 'party' as the first term."
assert set(stu_ans[1]) == set(
    reference_terms
), "Q3.2: Your system_rankings tuple is not a permutation of the permitted terms."  # must be a permutation of the official term set

# check spearmanr
assert isinstance(
    stu_ans[2], tuple
), "Q3.2: Your third element must be a tuple (of two floats). "
assert (
    len(stu_ans[2]) == 2
), "Q3.2: Your spearman output tuple should contain two floats."
assert isinstance(
    stu_ans[2][0], (float, np.floating)
), "Q3.2: Your spearman corr should be a float. "
assert isinstance(
    stu_ans[2][1], (float, np.floating)
), "Q3.2: Your spearman p-val should be a float. "


# Some hidden tests

del stu_ans

## Question 4: (30 points) Topic coherence.

One measure of topic model quality that is used e.g. to determine the optimal number of topics for a corpus is *topic coherence*. This is a measure of how semantically related the top terms in a topic model are. Topic models with low coherence tend to be filled with seemingly random words and hard to interpret, while high coherence usually indicates a clear semantic theme that's easily understood.

With their ability to represent word semantics, word embeddings are an ideal tool for computing topic coherence. In part 1, you'll implement a simple topic coherence function. In part 2, you'll apply that function to NMF topic modeling to find a setting for the number of topics that gives maximally coherent topic models.

We're going to use the same `text8_model` W2VTransformer object, which implements the word2vec embedding, that you loaded for the previous question.

### Part 4.1. (15 points) Average semantic distance as a text coherence measure.
Implement a function that takes a list of terms (strings) as input and returns a positive float indicating their semantic coherence. Here is the algorithm you should use:

1. For each input term, compute its word2vec embedding vector. One problem you might encounter is that some terms may not exist in the word2vec model. You get a "KeyError" exception when trying to transform that "out-of-vocabulary" term. You should ignore these terms: one way to do this is by wrapping your embedding call with a try/except statement that catches the KeyError and just ignores that word, and continues processing.

2. Once you have the list of embedding vectors for the input terms, compute their pairwise cosine similarity. If there are $n$ embedding vectors, this step will result in an $n x n$ matrix D.  If for some reason there are no input terms remaining (they are all out-of-vocabulary) just return 0.

3. Obviously the most similar word to a term is itself, indicated by a "1" on the diagonal of $D$. But we don't want those: we only care about the pairwise distances to *other* terms, so to deal that case, set the diagonal to zero.

4. Return the mean over all pairwise distances in D (with self-distances set to zero).  This is our simple coherence measure.

Be sure to try it out on some samples. For example, here's what our reference implementation returns:

`topical_coherence(['car', 'airplane', 'taxi', 'bus', 'vehicle', 'transport'])`

0.46063321000999874

`topical_coherence(['apple', 'banana', 'cherry', 'watermelon', 'lemon', 'orange'])`

0.43306025200419956

`topical_coherence(['possible', 'mean', 'volcano', 'feature', 'record', 'quickly'])`

0.1150558124192887

Your function should return the above measure of topic coherence for the following three lists, as a tuple of three corresponding floats:

`['train', 'car', 'bicycle', 'bus', 'vehicle', 'transport']`

`['scsi', 'drive', 'computer', 'storage', 'megabyte']`

`['introduction', 'pickle', 'guard', 'red', 'valiant']`


In [17]:
def answer_coherence_a():

    input_strings = [['train', 'car', 'bicycle', 'bus', 'vehicle', 'transport'], 
                     ['scsi', 'drive', 'computer', 'storage', 'megabyte'],
                     ['introduction', 'pickle', 'guard', 'red', 'valiant']]

    score_list = []

    for h in range(len(input_strings)):
        temp_list = []
        for i in range(len(input_strings[h])):
            try:
                vec = text8_model.transform(input_strings[h][i])
                temp_list.append(vec)
            except:
                print('not an available word')
        
        embeddings = np.vstack(temp_list)
        embedding_rows, embedding_columns = embeddings.shape

        similarity_array = np.zeros((embedding_rows, embedding_rows))
        for i in range(embedding_rows):
            for j in range(embedding_rows):
                sim = cosine_similarity(embeddings[i].reshape(1, -1), embeddings[j].reshape(1, -1)).item()
                similarity_array[i, j] = sim
        
        np.fill_diagonal(similarity_array, 0)
        final_score = np.mean(similarity_array)
        score_list.append(final_score)
        result = tuple(score_list)

    return result

In [18]:
stu_ans = answer_coherence_a()

assert isinstance(stu_ans, tuple), "Q4.1: Your function should return a tuple. "
assert (
    len(stu_ans) == 3
), "Q4.1: Your function should return a tuple of three elements. "

for i, item in enumerate(stu_ans):
    assert isinstance(
        item, (float, np.floating)
    ), f"Q4.1: Your answer at index {i} should be a float number. "


# Some hidden tests

del stu_ans

### Part 4.2 (15 points) Applying semantic coherence to topic model selection.

Now you'll use the semantic coherence measure you developed in Part 1 with topic models computed using Non-Negative Matrix Factorization.

Implement a simple loop that trains an NMF topic model, for number of topics **from 2 to 10 inclusive**. At each iteration, compute your topic coherence measure on the **top 10** words for each topic. Then compute the *median* topic coherence over all these topic scores.

Your function should return a list of 9 median coherence scores, corresponding to each choice of the number of topics to use with NMF.  Which choice gives the highest median semantic coherence?

When creating the NMF object, use these parameter settings: `random_state=42, init="nndsvd"`.  

In [19]:
### Use the following code to prepare input to the NMF topic model.
### It assumes you've loaded the 20newgroups variables at the beginning of this assignment
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer_NMF = TfidfVectorizer(
    max_features=20000,  # only top 5k by freq
    lowercase=True,  # drop capitalization
    ngram_range=(1, 1),
    min_df=2,  # note: absolute count of doc
    max_df=0.05,  # note: % of docs
    token_pattern=r"\b[a-z]{3,12}\b",  # remove short, non-word-like terms
    stop_words="english",
)  # default English stopwords

tfidf_documents_NMF = tfidf_vectorizer_NMF.fit_transform(documents_train)
feature_names_NMF = tfidf_vectorizer_NMF.get_feature_names()

In [20]:
from sklearn.decomposition import NMF
import statistics

def answer_coherence_b():
    
    score_list = []

    for i in range(2, 11):
    
        n_topics = i
        X = tfidf_documents_NMF

        nmf = NMF(n_components=n_topics, random_state=42, init="nndsvd")
        W = nmf.fit_transform(X) 
        H = nmf.components_

        top = 10
        topic_index_max = n_topics
    
        topic_list = []

        for topic_index in range(0, topic_index_max):
            top_indices = np.argsort(H[topic_index, :])[::-1]
            top_terms = []
        
            for term_index in top_indices[0:top]:
                top_terms.append(feature_names_NMF[term_index])
        
            topic_list.append(top_terms)
        
    
    
    
    
        input_strings = topic_list
        input_means = []

        for h in range(len(input_strings)):
            temp_list = []
            for i in range(len(input_strings[h])):
                try:
                    vec = text8_model.transform(input_strings[h][i])
                    temp_list.append(vec)
                except:
                    print('not an available word')
        
            embeddings = np.vstack(temp_list)
            embedding_rows, embedding_columns = embeddings.shape

            similarity_array = np.zeros((embedding_rows, embedding_rows))
            for i in range(embedding_rows):
                for j in range(embedding_rows):
                    sim = cosine_similarity(embeddings[i].reshape(1, -1), embeddings[j].reshape(1, -1)).item()
                    similarity_array[i, j] = sim
        
            np.fill_diagonal(similarity_array, 0)
            mean_score = np.mean(similarity_array)
            input_means.append(mean_score)
            
        median_score = statistics.median(input_means)
        score_list.append(median_score)
        result = score_list

    return result

In [21]:
stu_ans = answer_coherence_b()

assert isinstance(stu_ans, list), "Q4.2: Your function should return a list. "
assert (
    len(stu_ans) == 9
), "Q4.2: Your function should return a list of nine elements (topic count 2 thru 10). "

for i, item in enumerate(stu_ans):
    assert isinstance(
        item, (float, np.floating)
    ), f"Q4.2: Your answer at index {i} should be a float number. "


# Some hidden tests

del stu_ans